## Homework 3

#### 1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.


In [163]:
import pandas as pd
from functools import reduce
from plotly import graph_objects as go
import datetime as dt

In [81]:
#Load data 

users = pd.read_csv('Урок3.E-commerce/user_table.csv')
search_page = pd.read_csv('Урок3.E-commerce/search_page_table.csv')
payment_page = pd.read_csv('Урок3.E-commerce/payment_page_table.csv')
payment_confirmation = pd.read_csv('Урок3.E-commerce/payment_confirmation_table.csv')
home_page = pd.read_csv('Урок3.E-commerce/home_page_table.csv')

In [82]:
#Rename columns 


home_page = home_page.rename(
    columns={'page': 'home_page'}
)

search_page = search_page.rename(
    columns={'page': 'search_page'}
)

payment_page = payment_page.rename(
    columns={'page': 'payment_page'}
)

payment_confirmation = payment_confirmation.rename(
    columns={'page': 'payment_confirmation'}
)

In [83]:
#Connect datasets with merge

data = [users, home_page, search_page, payment_page, payment_confirmation]

df = reduce(lambda left, right: pd.merge(left, right, on='user_id', how='left'), data)

df.head(15)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN
5,993454,2015-03-03,Desktop,Male,home_page,NaN,NaN,NaN
6,377150,2015-04-15,Desktop,Female,home_page,search_page,NaN,NaN
7,137258,2015-01-27,Mobile,Male,home_page,NaN,NaN,NaN
8,608249,2015-04-12,Desktop,Female,home_page,NaN,NaN,NaN
9,615615,2015-02-24,Mobile,Female,home_page,search_page,NaN,NaN


In [84]:
# Decode event to 0 or 1

df['home_page'] = df['home_page'].apply(
    lambda x: 1 if x == 'home_page' else 0
)


df['search_page'] = df['search_page'].apply(
    lambda x: 1 if x == 'search_page' else 0
)

df['payment_page'] = df['payment_page'].apply(
    lambda x: 1 if x == 'payment_page' else 0
)

df['payment_confirmation'] = df['payment_confirmation'].apply(
    lambda x: 1 if x == 'payment_confirmation_page' else 0
)

In [85]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [88]:
#

conversion = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum()
conversion.head()

home_page               90400
search_page             45200
payment_page             6030
payment_confirmation      452
dtype: int64

In [108]:
conversion = conversion.reset_index()
conversion.head()

,level_0,index,0
0,0,home_page,90400
1,1,search_page,45200
2,2,payment_page,6030
3,3,payment_confirmation,452


In [147]:
df_male = df.groupby('sex')['home_page', 'search_page', 'payment_page', 'payment_confirmation'].sum()

df_male

<ipython-input-147-6a49fdf1cb11>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,home_page,search_page,payment_page,payment_confirmation
sex,,,,
Female,45075,22676,3100,241
Male,45325,22524,2930,211


In [148]:
df_male = df_male.unstack('sex')

df_male

                      sex   
home_page             Female    45075
                      Male      45325
search_page           Female    22676
                      Male      22524
payment_page          Female     3100
                      Male       2930
payment_confirmation  Female      241
                      Male        211
dtype: int64

In [149]:
df_male = df_male.unstack('sex').reset_index()

df_male

sex,index,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [150]:
df_male.columns.name = None
df_male = df_male.rename(columns={'index': 'Page'})

In [151]:

df_male

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [159]:
fig = go.Figure()

# Male part
fig.add_trace(go.Funnel(
    y=df_male['Page'],
    x=df_male['Male'],
    name='Male',
    constraintext='outside',
    textposition='inside',
    textinfo='value+percent previous',
    )
)

# Female part
fig.add_trace(go.Funnel(
    y=df_male['Page'],
    x=df_male['Female'],
    name='Female',
    textposition='auto',
    textinfo='value+percent previous',
   )
)

fig.update_layout(
    title='Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600
)

fig.show()

#### Создать 2 новые фичи на основе колонки “date”: месяц и день недели

In [171]:
# Create a month column

df['date']= pd.to_datetime(df['date'])

In [172]:
df['month']= df['date'].dt.month

In [179]:
# Create a month column

df['weekday'] = df['date'].dt.weekday

In [180]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,weekday
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,3


#### Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).


In [200]:
df.groupby(['weekday'])['payment_confirmation'].agg(
    count = 'sum').reset_index().sort_values(by='count', ascending = False)

,weekday,count
0,0,77
5,5,70
1,1,69
3,3,66
6,6,62
2,2,54
4,4,54


In [210]:
df.groupby(['month'])['payment_confirmation'].agg(
    count='sum').reset_index().sort_values(by='count', ascending = False)

,month,count
0,1,189
1,2,173
3,4,46
2,3,44


#### Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).


In [221]:
df.groupby(['sex', 'weekday'])['payment_confirmation'].agg(
    count = 'sum').unstack()

count                        
weekday     0   1   2   3   4   5   6
sex                                  
Female     43  40  30  31  26  36  35
Male       34  29  24  35  28  34  27